# Test Gemini vs Ensemble for MMLU


to do:

-ignore claude, add gemma, palm

In [ ]:
from pprint import pprint
from datetime import datetime
from datasets import load_dataset

from utils import Annotate

In [ ]:
seed =42
now = datetime.now().strftime("%Y%m%d")

In [ ]:
dataset = load_dataset("cais/mmlu", "all")

# # take a small sample for dev purposes
dataset = dataset['test'].shuffle(seed=seed).select(range(20))

# user provided data description
DESCRIPTION = """
This is a massive multitask test consisting of multiple-choice questions from various branches of knowledge.
The test spans subjects in the humanities, social sciences, hard sciences, and other areas that are important for some people to learn.
To attain high accuracy on this test, models must possess extensive world knowledge and problem solving ability.
This covers 57 subjects  across STEM, the humanities, the social sciences, and more. 
It ranges in difficulty from an elementary level to an advanced professional level, and it tests both world knowledge and problem solving ability. 
Subjects range from traditional areas, such as mathematics and history, to more specialized areas like law and ethics.
"""


In [ ]:
gemini_prompt_template = """
<QUESTION>
{datapoint}
</QUESTION>
------------

<CHOICES>
{labels}
</choices>
------------

INSTRUCTION:
- read the above question carefully.
- you are given 4 choices seperated by comma in <CHOICES>.
- take your time and pick the precise correct answer from <CHOICES> for the given <QUESTION>.
- remember that there is always only one correct answer.
- return the exact correct answer from <CHOICES>. Don't provide explanations.
"""

In [ ]:
prompt = [gemini_prompt_template.format(description= DESCRIPTION,
                                        datapoint=x['question'],
                                        labels=x['choices']) for x in dataset]
print(len(prompt))

In [ ]:
models = [
    # "palm",
    # "gemini",
    "claude"
    ]

ann = Annotate()


In [ ]:
r = await ann.classification(prompt, models=models)

In [ ]:
result = [dataset['choices'][idx][v] for idx, v in enumerate(dataset['answer'])]
pprint(result)

# debug GLAD

In [ ]:
from utils import glad

In [ ]:
tc = [20000, 20, 1000, 2, 0.5, 0.5]

In [ ]:
import json

with open('./data/output/annotation_output__20240515.json', 'r') as file:
    ddddd = json.load(file)

In [ ]:
glad(ddddd, tc)